In [10]:
# Install dependencies (run once per environment)
%pip install -q dspy python-dotenv


Note: you may need to restart the kernel to use updated packages.


In [11]:
# Basic imports and environment setup
import os
import dspy
from dotenv import load_dotenv
from dspy import History

# Load API keys from .env
load_dotenv()

# Configure model provider (OpenAI-only, per LangGraph agent)
lm = dspy.LM("openai/gpt-5-mini", api_key=os.getenv("OPENAI_API_KEY"), temperature=1, max_tokens=16000)

dspy.configure(lm=lm)

# Conversation memory
conversation_history = History(messages=[])

print("DSPy configured for Deep Research agent.")


DSPy configured for Deep Research agent.


In [12]:
# Utils: helpers
from datetime import datetime


def get_today_str() -> str:
    now = datetime.now()
    return f"{now:%a} {now:%b} {now.day}, {now:%Y}"


In [13]:
# Tools: OpenAI-only research + think tool
from typing import List, Literal


def openai_search(queries: List[str], max_results: int = 5, topic: Literal["general", "news"] = "general") -> str:
    """Generate comprehensive research responses using the model's knowledge base.

    Mirrors the LangGraph `openai_search` tool behavior at a high level.
    We keep a simple sequential loop for clarity in DSPy.
    """
    if not isinstance(queries, list) or not queries:
        return "No research results could be generated. Please provide queries."

    results = []
    for q in queries[:max_results]:
        if topic == "news":
            prompt = f"""Please provide a comprehensive research summary for the following current events query: "{q}"

Focus on recent developments and provide:
1. Key facts and developments
2. Timeline of important events
3. Current status and implications
4. Sources and references you know of

Be thorough and objective. Include dates where relevant."""
        else:
            prompt = f"""Please provide comprehensive information for the following research query: "{q}"

Structure your response to include:
1. Key facts and background information
2. Important details and context
3. Current understanding and implications
4. Any relevant examples or case studies

Be thorough and provide detailed, accurate information based on your knowledge."""
        # Single-turn call
        resp = dspy.Predict("answer")(question=prompt)  # lightweight call
        content = getattr(resp, "answer", "") if isinstance(resp, dspy.Prediction) else str(resp)
        results.append(f"--- RESEARCH RESULT: {q} ---\n{content}\n")

    if not results:
        return "No research results could be generated."
    return (f"OpenAI Research Results ({topic} focus):\n\n" + "\n\n".join(results)).strip()


def think_tool(reflection: str) -> str:
    """Strategic reflection tool for research planning.

    Use after searches to analyze results and plan next steps.
    """
    return f"Reflection recorded: {reflection}"


In [14]:
# ReAct Signature for Deep Research

class ResearchReActSignature(dspy.Signature):
    """
    You are a deep research assistant. Use tools to gather information (openai_search for research, think_tool for reflection).
    Keep searches focused, reflect after searches, and finish with a concise, well-structured research answer.

    You can call tools: openai_search, think_tool.
    When finished, produce:
    - `action`: the primary tool used (one of: openai_search, think_tool, answer_direct)
    - `tool_result`: the most relevant tool output you used (may be empty for answer_direct)
    - `answer`: the final research answer/report
    Keep responses clear and professional.
    """
    user_message: str = dspy.InputField(description="The user's research request")
    history: dspy.History = dspy.InputField(description="Conversation history")

    reasoning: str = dspy.OutputField(description="Brief plan and justification")
    action: str = dspy.OutputField(description="Chosen action/tool")
    tool_result: str = dspy.OutputField(description="Tool output used to answer")
    answer: str = dspy.OutputField(description="Final research answer")


In [15]:
# ReAct agent with research tools

react_agent = dspy.ReAct(
    ResearchReActSignature,
    tools=[openai_search, think_tool],
    max_iters=5,
)



In [16]:
# Entry point using ReAct agent

def run_deep_research(user_message: str) -> dict:
    if not isinstance(user_message, str) or not user_message.strip():
        return {"answer": "", "action": "reject", "tool_result": ""}

    conversation_history.messages.append({"role": "user", "content": user_message})

    result = react_agent(
        user_message=user_message,
        history=conversation_history,
    )

    answer = getattr(result, "answer", "")
    action = getattr(result, "action", "")
    tool_result = getattr(result, "tool_result", "")

    if isinstance(answer, str) and answer.strip():
        conversation_history.messages.append({"role": "assistant", "content": answer})

    return {"answer": answer, "action": action, "tool_result": tool_result}



In [17]:
# Examples / smoke tests (ReAct)

print("\n--- Example 1 ---")
resp = run_deep_research("Give me a concise overview of Rust ownership vs borrow checker with examples.")
print(resp)

print("\n--- Example 2 ---")
resp2 = run_deep_research("Summarize the latest approaches to structured reasoning in LLMs.")
print(resp2)



--- Example 1 ---
{'answer': 'Overview (short)\n- Ownership: Each value has one owner (a variable). When the owner goes out of scope, the value is dropped.\n- Move vs Copy: Non-primitive types (e.g., String, Vec) are moved by default. Types that implement Copy (integers, bools, char, simple tuples) are copied on assignment.\n- Borrowing: You can create references: &T (immutable borrow) or &mut T (mutable borrow). The borrow checker enforces safety rules at compile time.\n- Key borrow rules:\n  1. At any time, you may have either:\n     - any number of immutable (&T) references, or\n     - exactly one mutable (&mut T) reference.\n  2. References must always be valid (no dangling refs).\n- Lifetimes: The compiler tracks how long references live. You annotate explicit lifetimes when necessary for function signatures.\n\nExamples\n\n1) Move (ownership transfer) and Clone\nlet s1 = String::from("hello");\nlet s2 = s1; // s1 is moved into s2 and becomes invalid\n// println!("{}", s1); // er